In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, matthews_corrcoef, roc_auc_score, 
                             confusion_matrix, classification_report, roc_curve, auc)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

#TODO todays date?
# Account age reference point
REFERENCE_DATE = pd.Timestamp('2022-12-31').tz_localize('UTC')
# Paths
USER_FILE = '../data/twibot22/user.json'
LABEL_FILE = '../data/twibot22/label.csv'
SPLIT_FILE = '../data/twibot22/split.csv'

def load_data():
    print("Loading labels...")
    df_labels = pd.read_csv(LABEL_FILE)
    label_map = dict(zip(df_labels['id'].astype(str), 
                        (df_labels['label'] == 'bot').astype(int)))
    print(f"Loaded {len(label_map)} labels")
    
    print("Loading splits...")
    df_split = pd.read_csv(SPLIT_FILE)
    split_map = dict(zip(df_split['id'].astype(str), df_split['split']))
    print(f"Loaded {len(split_map)} splits")
    
    print("Loading user data...")
    with open(USER_FILE, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    
    data = []
    for i, entry in enumerate(raw_data):
        # Control print
        if (i + 1) % 50000 == 0:
            print(f"  Processing {i+1}/{len(raw_data)}...")
        
        user_id = str(entry.get('id', ''))
        if user_id not in label_map or user_id not in split_map:
            continue
        
        metrics = entry.get('public_metrics', {})
        
        age_days = 0
        try:
            dt = pd.to_datetime(entry.get('created_at'))
            if dt.tz is None: 
                dt = dt.tz_localize('UTC')
            age = (REFERENCE_DATE - dt).days
            age_days = max(age, 0)
        except: 
            pass
        
        img_url = entry.get('profile_image_url', '')

        # Raw features
        features = {
            'id': user_id,
            'followers_count': int(metrics.get('followers_count', 0)),
            'following_count': int(metrics.get('following_count', 0)),
            'tweet_count': int(metrics.get('tweet_count', 0)),
            'listed_count': int(metrics.get('listed_count', 0)),
            'username_length': len(entry.get('username', '')),
            'account_age_days': age_days,
            'verified': 1 if entry.get('verified') is True else 0,
            'default_profile_image': 1 if 'default_profile_images' in img_url else 0,
            'label': label_map[user_id],
            'split': split_map[user_id]
        }
        data.append(features)
    
    df = pd.DataFrame(data)
    print(f"Total: {len(df)} records")
    return df

df = load_data()
print(f"\nDataset splits:\n{df['split'].value_counts()}\nClass distribution:\n{df['label'].value_counts()}")

Loading labels...
Loaded 1000000 labels
Loading splits...
Loaded 1000000 splits
Loading user data...
  Processing 50000/1000000...
  Processing 100000/1000000...
  Processing 150000/1000000...
  Processing 200000/1000000...
  Processing 250000/1000000...
  Processing 300000/1000000...
  Processing 350000/1000000...
  Processing 400000/1000000...
  Processing 450000/1000000...
  Processing 500000/1000000...
  Processing 550000/1000000...
  Processing 600000/1000000...
  Processing 650000/1000000...
  Processing 700000/1000000...
  Processing 750000/1000000...
  Processing 800000/1000000...
  Processing 850000/1000000...
  Processing 900000/1000000...
  Processing 950000/1000000...
  Processing 1000000/1000000...
Total: 1000000 records

Dataset splits:
split
train    700000
val      200000
test     100000
Name: count, dtype: int64
Class distribution:
label
0    860057
1    139943
Name: count, dtype: int64


In [2]:
feature_cols = ['followers_count', 'following_count', 'tweet_count', 'listed_count',
                'username_length', 'account_age_days', 'verified', 'default_profile_image']



X_train = df[df['split'] == 'train'][feature_cols]
y_train = df[df['split'] == 'train']['label']
X_valid = df[df['split'] == 'val'][feature_cols]
y_valid = df[df['split'] == 'val']['label']
X_test = df[df['split'] == 'test'][feature_cols]
y_test = df[df['split'] == 'test']['label']

print(f"Train: {len(X_train)}, Valid: {len(X_valid)}, Test: {len(X_test)}")

print("\nTraining RandomForest...")
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=5,
                                  min_samples_leaf=2, class_weight='balanced', 
                                  random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

def evaluate(X, y, name):
    y_pred = rf_model.predict(X)
    y_proba = rf_model.predict_proba(X)[:, 1]
    
    metrics = {
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred, zero_division=0),
        'Recall': recall_score(y, y_pred, zero_division=0),
        'F1': f1_score(y, y_pred, zero_division=0),
        'MCC': matthews_corrcoef(y, y_pred),
    }
    
    print(f"\n {name}:")
    for metric, value in metrics.items():
        print(f"  {metric:12s}: {value:.4f}")
    
    return metrics, y_pred, y_proba

train_metrics, y_train_pred, y_train_proba = evaluate(X_train, y_train, "TRAIN")
valid_metrics, y_valid_pred, y_valid_proba = evaluate(X_valid, y_valid, "VALIDATION")
test_metrics, y_test_pred, y_test_proba = evaluate(X_test, y_test, "TEST")

joblib.dump(rf_model, '../models/twibot22_rf.joblib')
print("\n Model saved")

Train: 700000, Valid: 200000, Test: 100000

Training RandomForest...

 TRAIN:
  Accuracy    : 0.8979
  Precision   : 0.4235
  Recall      : 0.8567
  F1          : 0.5668
  MCC         : 0.5576

 VALIDATION:
  Accuracy    : 0.6667
  Precision   : 0.4223
  Recall      : 0.5226
  F1          : 0.4672
  MCC         : 0.2314

 TEST:
  Accuracy    : 0.6768
  Precision   : 0.4629
  Recall      : 0.6083
  F1          : 0.5257
  MCC         : 0.2936

 Model saved
